# Further updates to distributed river shapes
Cyril identified minor issues with the `maxup` field in split polygon cases, and `uparea` in the most downstream polygon of every case.

We already made changes to certain river shapefiles, and we should have every single one of them in the updates folder. We'll check this and if so proceed with processing.

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import xarray as xr

In [2]:
# Data location
cs_main_folder = Path("/scratch/gwf/gwf_cmt/wknoben/camels-spat-upload")
cs_update_folder = Path("/scratch/gwf/gwf_cmt/wknoben/camels-spat-upload-updates/")
cs_update_folder2 = Path("/scratch/gwf/gwf_cmt/wknoben/camels-spat-upload-updates-2/")

In [3]:
cs_meta = pd.read_csv(cs_update_folder2 / "camels-spat-metadata.csv")

## Check if we have what we think we have

In [4]:
dist_riv_shapes_in_updates = 0  # we expect to have a file for each basin here
lump_bas_shapes_in_updates = 0  # we expect to have a file for each basin here

for ix,row in cs_meta.iterrows():
    scale = row.subset_category
    cntry = row.Country
    statn = row.Station_id
    basin_id = f"{cntry}_{statn}"

    # First, check if we have a river shapefile in the updates folder 
    # Note: we know updates2 at this point only contains observation netcdf files
    path_middle_dist = f'shapefiles/{scale}/shapes-distributed'
    path_middle_lump = f'shapefiles/{scale}/shapes-lumped'
    
    riv_path = cs_update_folder / path_middle_dist / basin_id / f"{basin_id}_distributed_river.shp"
    bas_path = cs_update_folder / path_middle_lump / basin_id / f"{basin_id}_lumped.shp"

    if riv_path.exists():
        dist_riv_shapes_in_updates += 1
    else:
        #print(f"{riv_path.name} missing in updates folder")
        riv_path = cs_main_folder / path_middle_dist / basin_id / f"{basin_id}_distributed_river.shp"
        riv = gpd.read_file(riv_path)
        print(f"{riv_path.name} only found in old upload. Length = {len(riv)}; COMID = {riv['COMID'].values}")
    if bas_path.exists():
        lump_bas_shapes_in_updates += 1
    else:
        print(f"{riv_path.name} missing in updates folder")

CAN_01AK006_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_01AL004_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_01BU009_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_01CD005_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_01EE005_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_01FJ002_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_02BA005_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_02CG003_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_02GA043_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_02YA002_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_02YK008_distributed_river.shp only found in old upload. Length = 1; COMID = [None]
CAN_02YL005_distributed_river.shp only foun

In [5]:
print(f"{dist_riv_shapes_in_updates} in updates folder")
print(f"{lump_bas_shapes_in_updates} in updates folder")

1343 in updates folder
1426 in updates folder


Prints above indicate that for the 83 basins missing in the updates folder we have no polygons anyway, so these can be left as is.

## Processing

In [169]:
def update_nextdownid_and_up(basin_id, gdf, comid, new_ndid):

    '''Fixes connectivity in a number of specific cases where we have back-to-back nested polygons'''
    print(f"Applying manual update to {basin_id}")

    # First fix the NextDownID
    mask = river['COMID'] == comid
    river.at[river.index[mask][0], 'NextDownID'] = new_ndid
    assert river.loc[mask, 'NextDownID'].iloc[0] == new_ndid, "Update failed: NextDownID was not set correctly"
    print(f" - NextDownID update to {new_ndid} complete")

    # Now check if this downstream COMID already has the correct up ID in one of its up columns
    mask = river['COMID'] == new_ndid
    assert mask.sum() == 1, f"Expected to identify 1 polygon with COMID {new_ndid}, but found {mask.sum()}"
    if comid in river.loc[mask, ['up1','up2','up3','up4']].values:
        print(f" - Upstream COMID {comid} already found in up values {river.loc[mask, ['up1','up2','up3','up4']].values}")
    else:
        #print(f" - COMID {comid} not already present in any upstream values: {river.loc[mask, ['up1','up2','up3','up4']].values[0]}. Trying to identify old un-split COMID")
        up_mask = np.floor(comid) == river.loc[mask, ['up1','up2','up3','up4']].values[0]
        if up_mask.sum() == 1:
            
            # identify the correct up column
            up_name = f"up{np.where(up_mask)[0][0]+1}"
            #print(f" - Original COMID {np.floor(comid)} found in up values {river.loc[mask, ['up1','up2','up3','up4']].values[0]}. Replacing {up_name} with {comid}")
            assert (float(comid) - float(river.loc[mask, up_name].iloc[0]) > 0) & (float(comid) - float(river.loc[mask, up_name].iloc[0]) < 1), f"COMID = {comid}; current up = {float(river.loc[mask, up_name].iloc[0])}"

            # get the index of this row so we can use .at[]
            river.at[river.index[mask][0], up_name] = comid
            assert river.loc[mask, up_name].iloc[0] == comid, f"Update failed: {up_name} was not set correctly"
            print(f" - {up_name} update to {comid} complete")
        else:
            # COMID not found, or more than once
            print(f"Original COMID {np.floor(comid)} not uniquely identified in up values {river.loc[mask, ['up1','up2','up3','up4']].values[0]}. Check manually.")

    return river

In [8]:
def update_nextdownid(basin_id, gdf, comid, new_ndid):
    print(f"Applying manual update to {basin_id}")
    river.loc[river['COMID'] == comid, 'NextDownID'] = new_ndid
    assert river.loc[river['COMID'] == comid, 'NextDownID'].iloc[0] == new_ndid, "Update failed: NextDownID was not set correctly"
    return river

In [9]:
def find_downstream_segment(gdf):
    possibly_downstream = []
    for ix, row in gdf.iterrows():
        nextDownID = row['NextDownID']
        if not nextDownID in gdf['COMID'].values:
            possibly_downstream.append(row['COMID'])
    return possibly_downstream        

In [10]:
def add_missing_segment(basin_id, river, basin):
    '''Adds a river segment to the river shapefile in cases where we had a small nested basin that didn't have its own river segment.
       This happens in cases where the nested basin is either off the side of the main MERIT river network, or upstream of where the
       MERIT network begins to be delineated.
       
       Based on a dry run of the main loop we found 12 cases where the basin shapefile contains exactly 1 entry more than the river,
       and 1 case where the basin shapefile contains 7 extra entries. This function adds that missing row to the river shapefile.
       
       Summary based on visual inspection:
       - CAN_01AL002:  nested gauge to the side of the main river stem; nested gauge is part of CAMELS-SPAT (CAN_01AL004)
       - CAN_01BU002:  nested gauge upstream of where a MERIT river starts; nested gauge is part of CAMELS-SPAT (CAN_01BU009)
       - CAN_02BF001:  ! nested gauges to upstream & sideways of where a MERIT river start; none are part of CAMELS-SPAT because no sub-daily streamflow was available
                         (CAN_02BF005, CAN_02BF006, CAN_02BF007, CAN_02BF008, CAN_02BF009, CAN_02BF012, CAN_02BF0013)
       - CAN_02GA010:  nested gauge to the side of the main river stem, nested gauge is part of CAMELS-SPAT (CAN_02GA043)
       - CAN_05DA007:  ! nested gauge upstream of where a MERIT river starts; nested gauge is not part of CAMELS-SPAT because no sub-daily streamflow was available (CAN_05DA010)
       - CAN_05DA009:  same case as CAN_05DA007, just for a bigger basin that includes CAN_05DA007
       - CAN_05EE009:  nested gauge to the side of the main river stem, nested gauge is part of CAMELS-SPAT (CAN_05EE006)
       - CAN_05LL015:  nested gauge upstream of where a MERIT river starts; nested gauge is part of CAMELS-SPAT (CAN_05LL027)
       - CAN_08HA010:  nested gauge to the side of the main river stem, nested gauge is part of CAMELS-SPAT (CAN_08HA068)
       - USA_01435000: nested gauge upstream of where a MERIT river starts; nested gauge is part of CAMELS-SPAT (USA_01434025)
       - USA_01543000: nested gauge upstream of where a MERIT river starts; nested gauge is part of CAMELS-SPAT (USA_01542810)
       - USA_01543500: same case as USA_01543000, just for a bigger basin that includes USA_01543000
       - USA_14306500: nested gauge upstream of where a MERIT river starts; nested gauge is part of CAMELS-SPAT (USA_14306340)
       '''

    # Check we're looking at a known case
    if not basin_id in ['CAN_01AL002',  'CAN_01BU002',  'CAN_02BF001', 'CAN_02GA010', 'CAN_05DA007',
                        'CAN_05DA009',  'CAN_05EE009',  'CAN_05LL015', 'CAN_08HA010', 'USA_01435000',
                        'USA_01543000', 'USA_01543500', 'USA_14306500']:
        print(f"add_missing_segment(): case {basin_id} not implemented. Early exit.")
        return

    if basin_id == 'CAN_02BF001':  # case with 7 missing fields; simpler to just add the stuff manually

        uparea = [                                                        # COMID:
            basin[basin['COMID'] == 72044865.2]['unitarea'].values[0] +      # 72044865.2 (itself, and everything else upstream)
                basin[basin['COMID'] == 72044865.3]['unitarea'].values[0] +      
                basin[basin['COMID'] == 72044865.4]['unitarea'].values[0] +
                basin[basin['COMID'] == 72044865.5]['unitarea'].values[0] +
                basin[basin['COMID'] == 72044865.6]['unitarea'].values[0] +      
                basin[basin['COMID'] == 72044865.7]['unitarea'].values[0] +  
                basin[basin['COMID'] == 72044865.8]['unitarea'].values[0],
            basin[basin['COMID'] == 72044865.3]['unitarea'].values[0] +      # 72044865.3 (itself + upstream .4, .5, .6, .7, .8)
                basin[basin['COMID'] == 72044865.4]['unitarea'].values[0] +
                basin[basin['COMID'] == 72044865.5]['unitarea'].values[0] +
                basin[basin['COMID'] == 72044865.6]['unitarea'].values[0] +      
                basin[basin['COMID'] == 72044865.7]['unitarea'].values[0] +  
                basin[basin['COMID'] == 72044865.8]['unitarea'].values[0],
            basin[basin['COMID'] == 72044865.4]['unitarea'].values[0],       # 72044865.4 (just itself, has no upstream)
            basin[basin['COMID'] == 72044865.5]['unitarea'].values[0] +      # 72044865.5 (itself + upstream .6, .7, .8
                basin[basin['COMID'] == 72044865.6]['unitarea'].values[0] +      
                basin[basin['COMID'] == 72044865.7]['unitarea'].values[0] +  
                basin[basin['COMID'] == 72044865.8]['unitarea'].values[0],
            basin[basin['COMID'] == 72044865.6]['unitarea'].values[0] +      # 72044865.6 (itself + upstream .7, .8)
                basin[basin['COMID'] == 72044865.7]['unitarea'].values[0] +  
                basin[basin['COMID'] == 72044865.8]['unitarea'].values[0],   
            basin[basin['COMID'] == 72044865.7]['unitarea'].values[0] +      # 72044865.7 (itself + upstream .8)
                basin[basin['COMID'] == 72044865.7]['unitarea'].values[0],   
            basin[basin['COMID'] == 72044865.8]['unitarea'].values[0],       # 72044865.8 (just itself, has no upstream)
        ]
    
        new_segments = {'COMID'     : [72044865.2, 72044865.3, 72044865.4, 72044865.5, 72044865.6, 72044865.7, 72044865.8],
                        'NextDownID': [72044865.1, 72044865.2, 72044865.3, 72044865.3, 72044865.5, 72044865.6, 72044865.7],
                        'up1'       : [72044865.3, 72044865.4, 0,          72044865.6, 72044865.7, 72044865.8, 0         ],
                        'up2'       : [0,          72044865.5, 0,          0,          0,          0,          0         ],
                        'up3'       : [0,          0,          0,          0,          0,          0,          0         ],
                        'up4'       : [0,          0,          0,          0,          0,          0,          0         ],
                        'new_len_km': [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ],
                        'slope'     : [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ],
                        'uparea'    : uparea,
                        'lengthdir' : [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ],
                        'sinuosity' : [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ],
                        'order'     : [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ],
                        'strmDrop_t': [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ],
                        'slope_taud': [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ],
                        'maxup'     : [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ],
                        'geometry'  : [np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan,     np.nan    ]
              }

        river = pd.concat([river, pd.DataFrame(new_segments)], ignore_index=True)

    else:  # case with 1 missing field
    
        # Find the comid that's in the basin but not the river
        b_ids = basin['COMID']
        r_ids = river['COMID']
        missing = b_ids[~b_ids.isin(r_ids)]
        assert len(missing) == 1, f"unexpected different number of missing basins in river file (expected 1, found {len(missing)}"
    
        # Find the downstream COMID (i.e., the one that's pointing to this as its upstream)
        mask = river[['up1', 'up2', 'up3', 'up4']].isin([missing.iloc[0]]).any(axis=1)
        assert mask.sum() == 1, f"expected to identify 1 COMID that has our current one as upstream, but found {len(mask)}"
    
        # Check we have a basin area
        uparea = basin[basin['COMID'] == missing.iloc[0]]['unitarea']
        assert len(uparea) == 1, f"expected to identify 1 COMID to use for upstream area, but found {len(uparea)}"
    
        # Update the geodataframe
        new_segment = {'COMID'     : missing.iloc[0],
                       'NextDownID': river[mask]['COMID'].iloc[0],
                       'up1'       : 0.0,
                       'up2'       : 0.0,
                       'up3'       : 0.0,
                       'up4'       : 0.0,
                       'new_len_km': np.nan,
                       'slope'     : np.nan,
                       'uparea'    : uparea.iloc[0],
                       'lengthdir' : np.nan,
                       'sinuosity' : np.nan,
                       'order'     : np.nan,
                       'strmDrop_t': np.nan,
                       'slope_taud': np.nan,
                       'maxup'     : np.nan,
                       'geometry'  : np.nan
                      }
        river = pd.concat([river, pd.DataFrame([new_segment])], ignore_index=True)
    
        # Check we have the same values in either geodataframe now
        b_ids = basin['COMID']
        r_ids = river['COMID']
        missing = b_ids[~b_ids.isin(r_ids)]
        assert len(missing) == 0, f"still missing a basin: {missing}"

    return river

In [333]:
# Step 1: Create the reverse lookup graph: COMID -> list of upstream COMIDs
def build_upstream_graph(river):
    graph = defaultdict(list)
    upstream_cols = ['up1', 'up2', 'up3', 'up4']
    
    for _, row in river.iterrows():
        current = row['COMID']
        upstreams = pd.to_numeric(row[upstream_cols]).dropna()
        for up in upstreams:
            graph[current].append(up)
    return graph

# Step 2: Traverse upstream recursively (DFS) to collect all upstream COMIDs
def find_all_upstreams(graph, start, visited=None):
    if visited is None:
        visited = set()
    for upstream in graph.get(start, []):
        if upstream not in visited:
            visited.add(upstream)
            find_all_upstreams(graph, upstream, visited)
    return visited

# Step 3: Build the full lookup table for all COMIDs
def build_upstream_lookup(river):
    graph = build_upstream_graph(river)
    lookup = dict()
    for comid in river['COMID']:
        lookup[comid] = list(find_all_upstreams(graph, comid))
    return lookup

In [299]:
# make a temporary plotting dir
plot_dir = Path('./img')
plot_dir.mkdir(parents=True, exist_ok=True)

In [431]:
uparea_diffs = 0
uparea_updates = []
any_maxup_updated = 0

for ix,row in cs_meta.iterrows():
    scale = row.subset_category
    cntry = row.Country
    statn = row.Station_id
    basin_id = f"{cntry}_{statn}"

    # Paths
    path_middle_dist = f'shapefiles/{scale}/shapes-distributed/{basin_id}'
    path_middle_lump = f'shapefiles/{scale}/shapes-lumped/{basin_id}'
    
    riv_path = cs_update_folder / path_middle_dist / f"{basin_id}_distributed_river.shp"
    bas_path = cs_main_folder   / path_middle_dist / f"{basin_id}_distributed_basin.shp"
    lum_path = cs_update_folder / path_middle_lump / f"{basin_id}_lumped.shp"

    if riv_path.exists():  # i.e., if we have a file in the updates folder, and thus an actual polygon for this place
        
        # Load the shapefiles
        river = gpd.read_file(riv_path)
        basin = gpd.read_file(bas_path)
        lumped= gpd.read_file(lum_path)

        # --- CONNECTIVITY FIXES ---
        # For 7 specific cases, apply a manual fix to niche cases where two split polygons are directly upstream/downstream
        #  from one another, and the general code did not handle this correctly
        if basin_id == 'CAN_02ED003':  # Split polygon 72052405.1 currently has NextDownID 72051937.0 but this should be 72051937.2
            river = update_nextdownid_and_up(basin_id, river, 72052405.1, 72051937.2)  
        elif basin_id == 'CAN_02ED101':  # Exact case of CAN_02ED003, but smaller basin (i.e., the other one includes this one)
            river = update_nextdownid_and_up(basin_id, river, 72052405.1, 72051937.2)
        elif basin_id == 'CAN_05DA007':  # Split polygon 71032880.1 currently has NextDownID 71029794.0 but this should be 71029794.2
            river = update_nextdownid_and_up(basin_id, river, 71032880.1, 71029794.2)
        elif basin_id == 'CAN_05DA009':  # Exact case of CAN_05DA007, but larger basin (i.e., this one includes the other)
            river = update_nextdownid_and_up(basin_id, river, 71032880.1, 71029794.2)
        elif basin_id == 'CAN_08EE003':  # Split polygon 78004244.1 currently has NextDownID 78004211.0 but this should be 78004211.2
            river = update_nextdownid_and_up(basin_id, river, 78004244.1, 78004211.2)
        elif basin_id == 'CAN_08EE004':  # Exact case of CAN_05DA007, but larger basin (i.e., this one includes the other)
            river = update_nextdownid_and_up(basin_id, river, 78004244.1, 78004211.2)
        elif basin_id == 'CAN_08LD001':  # Split polygon 78010211.1 currently has NextDownID 78010204.0 but this should be 78010204.2
            river = update_nextdownid_and_up(basin_id, river, 78010211.1, 78010204.2)

        # For 13 basins, apply a fix to account for (a) missing stream segment(s)
        if len(river) != len(basin):
            #print(f"{scale}/{basin_id}: number of subbasins ({len(basin)}) does not match number of stream segments ({len(river)})")  # used this to identify the cases we check for in the function
            river = add_missing_segment(basin_id, river, basin)

        # --- UPAREA ---
        # Identify a new best guess at upstream areas and compare to what we have. Flag cases where differences are large and create a plot for those
        # Build the network table
        upstream_lookup = build_upstream_lookup(river)
        upstream_df = pd.DataFrame([
            {'COMID': k, 'all_upstream': v}
            for k, v in upstream_lookup.items()
        ])
        
        # Re-calculate upstream areas for each COMID
        comids = []
        uparea = []
        for ix, row in upstream_df.iterrows():
            comid = row['COMID']
            comids.append(comid)
            sum_ids = row['all_upstream'] + [comid]
            uparea.append(basin[basin['COMID'].isin(sum_ids)]['unitarea'].sum())
        check = pd.DataFrame(data={'COMID': comids,
                                   'uparea': uparea})
        
        # Combine with original river dataframe
        merged_df = river.merge(check, on='COMID', suffixes=('_org','_new'))
        merged_df['uparea_org'] = pd.to_numeric(merged_df['uparea_org'])
        merged_df['area_diff'] = np.where(abs(merged_df['uparea_org'] - merged_df['uparea_new']) > 1e-3,  # if any area differences larger than 0.001 km^2
                                          merged_df['uparea_org'] - merged_df['uparea_new'],  # track the actual area difference
                                          np.nan)  # otherwise put nan
        
        # Check if we have any large-ish discrepencies. If so, increment counter and plot
        if any(merged_df['area_diff'].notna()):
            uparea_diffs += 1

            # Create a plot - NOTE: I check these during a dry run and it all seems to make sense
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")  # shapely deprecation warning
                fig, ax = plt.subplots(1,3, figsize=(20,5))
            
                merged_df.plot(ax=ax[0],column='uparea_org',legend=True)
                ax[0].set_title('old')
                
                merged_df.plot(ax=ax[1], color='lightgrey')
                merged_df.plot(ax=ax[1],
                               column='area_diff',
                               legend=True,
                            )
                ax[1].set_title(f'{basin_id}: upstream area diff [km^2]')
            
                merged_df.plot(ax=ax[2],column='uparea_new',legend=True)
                ax[2].set_title('new')
                
                plt.savefig(plot_dir/f"{basin_id}_uparea_diffs.png", dpi=100)
                plt.close()

            # --- Update the river geodataframe
            # Make a selection of what to update
            update_threshold = 1e-3 # km^2
            update_this = merged_df[abs(merged_df['area_diff']) > update_threshold][['COMID','uparea_new', 'area_diff']].copy().reset_index(drop=True)
            update_this['basin'] = basin_id
            uparea_updates.append(update_this)  # save to csv later so we have a record of changes
            
            # Set indices and update
            river_tmp = river.copy()
            river_tmp = river_tmp.set_index('COMID')
            update_this = update_this.set_index('COMID')
            river_tmp['uparea'].update(update_this['uparea_new'])
            
            # Check that we did in fact update something
            assert len(river_tmp) == len(river), f"length of river dataframe changed during update (was {len(river)}; now {len(river_tmp)})"
            assert any(river_tmp['uparea'].values != river['uparea'].values), "no changed values found"
            
            # Re-assign the updated shapefile to 'river'
            river = river_tmp.reset_index().copy()

        # --- MAXUP ---
        has_up = (river[['up1','up2','up3','up4']] > 0.0).sum(axis=1)  # check if this row has any upstream segments defined
        up_sum = np.where(has_up, has_up, 0)  # count how many up segments each row has, and put in 0 if it hasn't got any (should account for NaN)
        if any(river['maxup'] != up_sum):
            any_maxup_updated += 1
            river['maxup'] = np.where(river['maxup'] != up_sum, up_sum, river['maxup'])  # replace existing maxup with new where different

        # --- save
        # we'll save the files regardless of updates, and just upload the whole thing
        des_fold = cs_update_folder2 / path_middle_dist
        des_fold.mkdir(exist_ok=True, parents=True)
        des_path = des_fold / f"{basin_id}_distributed_river.shp"
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")  # geopandas FutureWarning
            river.to_file(des_path)

# save the uparea updates to CSV
uparea_updates_df = pd.concat(uparea_updates)
uparea_updates_df = uparea_updates_df[['basin','COMID','uparea_new','area_diff']]
uparea_updates_df.to_csv('river_updates_upareas.csv', index=False)

Applying manual update to CAN_02ED003
 - NextDownID update to 72051937.2 complete
 - COMID 72052405.1 not already present in any upstream values: [72052405. 72053278.        0.        0.]. Trying to identify old un-split COMID
 - Original COMID 72052405.0 found in up values [72052405. 72053278.        0.        0.]. Replacing up1 with 72052405.1
 - up1 update to 72052405.1 complete
Applying manual update to CAN_02ED101
 - NextDownID update to 72051937.2 complete
 - COMID 72052405.1 not already present in any upstream values: [72052405. 72053278.        0.        0.]. Trying to identify old un-split COMID
 - Original COMID 72052405.0 found in up values [72052405. 72053278.        0.        0.]. Replacing up1 with 72052405.1
 - up1 update to 72052405.1 complete
Applying manual update to CAN_05DA007
 - NextDownID update to 71029794.2 complete
 - COMID 71032880.1 not already present in any upstream values: [71029920. 71032880.        0.        0.]. Trying to identify old un-split COMID
 - 

In [432]:
uparea_diffs, any_maxup_updated

(1039, 389)